in this notebook we implement the replication package by authors of static bug detection paper with LLM 
we make modification to their code seeking to address llm model unresponsive issues and allow code to continue to run after being interrupted.
basically we break the gptsa class down because however they did originally was brittle and make little sense and data driven in the worst way possible. 

All prereqs 

In [3]:
%pip install langchain_ollama
%pip install langchain
%pip install langchain_core

%pip install rich
%pip install scikit-learn

%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached scikit_learn-1.6.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached scipy-1.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.3 MB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import signal
import time
import json
import pickle
import sys
import os

#from langchain.llms import OpenAI
from langchain_ollama import ChatOllama

from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
    PromptTemplate,
)

from langchain.chains import LLMChain

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
#from langchain_core.output_parsers import StructuredOutputParser
from langchain_core.output_parsers import StrOutputParser



from rich import print
from sklearn.model_selection import KFold
from pathlib import Path

import os
import pandas as pd
import numpy as np

import random

import sys

import re

In [3]:
dataset_path = "/home/hhjww/Downloads/effectivenessPackage/src/data/dataset-all.csv"
output_path = "/home/hhjww/Downloads/effectivenessPackage/src/results"

tempsave_path = "/home/hhjww/Downloads/effectivenessPackage/src/caching/"
K = 5
number_of_folds = K

In [4]:
#INIT


__seed_value = 42
np.random.seed(__seed_value)
random.seed(__seed_value)
pd.options.display.max_colwidth = 100000

# loading the dataset if provided in the constructor's arguemnts
dataset_df = pd.read_csv(dataset_path).drop(["file", "procedure", "line"], axis=1)
dataset = (
    dataset_df.drop(["source"], axis=1)
    if ("source" in list(dataset_df.columns))
    else dataset_df
)

# group the dataset by its bug types
# this is done later cuz its data driven.

#create Kfold
# this is done in caching section u will know why 


# initiate LLM models
deepseek1b = ChatOllama(
    model="deepseek-r1:1.5b"
)
inference_model = ChatOllama(
    model="deepseek-r1:1.5b"
)



next we will produce the examples and validation data
cache method: take bug type / shots as input 

In [5]:
def get_validation_data(self, fold_index, buggy_folds, clean_folds):
    """
    returns [val_record_idx][key : (snippet, warning, target)]
    """
    validation_examples = []

    buggy_val_data = buggy_folds[fold_index]["val"]
    clean_val_data = clean_folds[fold_index]["val"]

    # buggy_train_data = buggy_folds[fold_index]["train"]
    # clean_train_data = clean_folds[fold_index]["train"]

    samples = pd.concat([buggy_val_data, clean_val_data])

    for index, val in samples.iterrows():
        snippet = val["snippet"]
        warning = val["warning"]
        target = val["buggy"]

        # entry = (snippet, warning, target)
        entry = {"snippet": snippet, "warning": warning, "target": target}
        validation_examples.append(entry)

    return validation_examples


In [6]:
def save_results(output_df, experiment, bug_type, strategy, model_name, fold):
    file_name = f"{experiment}_{bug_type}_{strategy}_{model_name}__fold{fold}.csv"
    results_output_dir = "/home/hhjww/Downloads/effectivenessPackage/src/results"


    if not os.path.isdir(results_output_dir):
        os.makedirs(results_output_dir)

    output_path = f"{results_output_dir}/{file_name}"
    output_df.to_csv(output_path, index=False)

In [7]:
def cache_data(bug_type_master, strategy): 
    # folds are based on bug type because the fold creation has data balancing 
    # zero shot has no examples 
    # one shot and few shot prompts implemented differently so each 
    
    #basic caches, data drive. 

    #we dont need to cache bug types because we can just get it??
    #this just get the bug types for the function 
    # split data by bug type then bugginess
    dfs_by_bug_type = {}
    grouped = dataset_df.groupby("bug_type")
    for bug_type, group_df in grouped:
        dfs_by_bug_type[bug_type] = {
            "all": group_df,
            "buggy": group_df[group_df["buggy"] == 1].reset_index(drop=True),
            "clean": group_df[group_df["buggy"] == 0].reset_index(drop=True),
        }
    datasets_by_bug_type = dfs_by_bug_type
    #done 

    #cache kfolds
    # we can just cache kfolds all in one file for both bug types regardless
    folds = {}
    if os.path.exists(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/folds.pkl"):
        #this load loads it for this function only. this is so that later we can cache the examples.
        print(f"found /home/hhjww/Downloads/effectivenessPackage/src/caching/folds.pkl")
        with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/folds.pkl", 'rb') as f:
            folds = pickle.load(f)
        
    else: 
        kf = KFold(n_splits=K, shuffle=True, random_state=__seed_value)
        folds = {}
        for bug_type in datasets_by_bug_type:
            buggy_df = datasets_by_bug_type[bug_type]["buggy"]
            buggy_folds = []
            for fold, (train_idx, val_idx) in enumerate(kf.split(buggy_df)):
                train_data = buggy_df.iloc[train_idx].reset_index()
                val_data = buggy_df.iloc[val_idx].reset_index()
                buggy_folds.append({"train": train_data, "val": val_data})

            clean_df = datasets_by_bug_type[bug_type]["clean"]
            clean_folds = []
            for fold, (train_idx, val_idx) in enumerate(kf.split(clean_df)):
                train_data = clean_df.iloc[train_idx].reset_index()
                val_data = clean_df.iloc[val_idx].reset_index()
                clean_folds.append({"train": train_data, "val": val_data})

            folds[f"{bug_type}_buggy"] = buggy_folds
            folds[f"{bug_type}_clean"] = clean_folds
        
        with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/folds.pkl", 'wb') as f:
            pickle.dump(folds, f)
            print(f"cached /home/hhjww/Downloads/effectivenessPackage/src/caching/folds.pkl")
            
    #end caching folds

    # cache validation data for each fold 
    # cached is bug type specific 
    validation_data_list = [0]*number_of_folds  
    if os.path.exists(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/validation_data_list.pkl"):
            #this load loads it for this function only. this is so that later we can cache the examples.
            print(f"loaded /home/hhjww/Downloads/effectivenessPackage/src/caching/validation_data_list.pkl")
            with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/validation_data_list.pkl", 'rb') as f:
                validation_data_list = pickle.load(f)

    else: 
        buggy_folds = folds[f"{bug_type_master}_buggy"]
        clean_folds = folds[f"{bug_type_master}_clean"]

        for fold in range(number_of_folds):
            #validation_data = get_validation_data(
            #    fold, buggy_folds, clean_folds
            #)
            validation_examples = []

            buggy_val_data = buggy_folds[fold]["val"]
            clean_val_data = clean_folds[fold]["val"]

            # buggy_train_data = buggy_folds[fold_index]["train"]
            # clean_train_data = clean_folds[fold_index]["train"]

            samples = pd.concat([buggy_val_data, clean_val_data])

            for index, val in samples.iterrows():
                snippet = val["snippet"]
                warning = val["warning"]
                target = val["buggy"]

                # entry = (snippet, warning, target)
                entry = {"snippet": snippet, "warning": warning, "target": target}
                validation_examples.append(entry)
                
            validation_data_list[fold] = validation_examples
        #end for
        with open('/home/hhjww/Downloads/effectivenessPackage/src/caching/validation_data_list.pkl', 'wb') as file:
            pickle.dump(validation_data_list, file)
            print(f"cached /home/hhjww/Downloads/effectivenessPackage/src/caching/validation_data_list.pkl")
    # end of caching validation data

    # prompt templates 
    if bug_type_master == "NULL_DEREFERENCE":
        targets = ResponseSchema(
            name="targets",
            description="Give me the list of all potential objects, method calls, and function calls that are suspected to be the cases of null dereference bugs detected by the analysis. Make sure the given potential suspects are the ones that their values are not being checked for being null or not in the code. The list should be in a format like this: [a, b, c, d]. If there weren't any potential suspects, you should return an empty list like this: [], which means that there is no cases of null dereference bug",
        )
        results = ResponseSchema(
            name="result",
            description="What is the result of the analysis? Is the given Java code buggy with a potential null dereference bug? Answer True if yes, Answer False if no.",
        )
        explanation = ResponseSchema(
            name="explanation",
            description='Any explanation you have for your decision. Explain your answer briefly. Make sure the text in the explanation escape double quotes with \. For example: "example"',
        )
        response_schemas = [targets, results, explanation]
        output_parser = StructuredOutputParser.from_response_schemas(
            response_schemas
        )
        format_instructions = output_parser.get_format_instructions()
        system_prompt = f"""
            You are an advanced static analyzer for programs and source codes written in Java. Your duty is to detect null dereference bugs in the given Java codes delimited by ####. Each of this Java codes contain a method implementation, and you are supposed to analyze the he body this method. For example, if the given Java code is like: "final static void FooBar(){{...}}", you only look and analyze the issues surrounded by curly braces and the codes inside of them.

            REMEMBER: Please make sure that if any objects and method or function calls and invokations could potentially return a null value and become a potential null dereference bug, please report them and consider the given Java code as a buggy code with null dereference issue. 

            REMEMBER: If the value of the objects or the result of the method and function calls and invokations are being checked with if statements for being null or not, this means that the null dereference bug is handled, and this is not a case of null dereference bug, and those objects and method or function calls and invokations are not buggy.

            REMEMBER: Only report the objects or method and function calls and invokations that you are sure and confident that they are high likely a null dereference potential bug. Do not report if you are not sure.

            Use the following formatting guideline for the output:

            << FORMATTING >>
            {format_instructions}
        """
        warning_inference_prompt = (
            lambda warning: f"""
            Which object or method call or function call could be null null in the following warning message delimited by @@@@?
            
            @@@@
            {warning}
            @@@@
            
            Please use the following output format for reporting the object:
            The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

            ```json
            {{
                "targets": ["OBJECT_NAME"],
                "result": "True",
                "explanation": "The object OBJECT_NAME could potentially be null and dereferenced later"
            }}
            ```
            
            REMEMBER: In case instead of an object, a method call or function call was the suspect, you need to follow the following output format:
            
            ```json
            {{
                "targets": ["METHOD_INVOKATION"],
                "result": "True",
                "explanation": "The call to METHOD_INVOKATION could potentially return null and dereferenced later"
            }}
            ```
            
            """
        )

    elif bug_type_master == "RESOURCE_LEAK":
        targets = ResponseSchema(
            name="targets",
            description="Give me the list of all potential resources and allocations that are suspected to be the cases of resource leakage bugs detected by the analysis. Make sure the detected potential suspects are the resources that are acquired and not released afterward by the end of the method. The list should be in a format like this: [a, b, c, d]. If there weren't any potential suspects, you should return an empty list like this: [], which means that there is no cases of resource leakage bug",
        )
        result = ResponseSchema(
            name="result",
            description="What is the result of the analysis? Is the given Java code buggy with a resource leakage bug? Answer True if yes, Answer False if no.",
        )
        explanation = ResponseSchema(
            name="explanation",
            description="Any explanation you have for your decision. Explain your answer briefly. Make sure the text in the explanation is JSON parsable or JSON friendly",
        )
        response_schemas = [targets, result, explanation]
        output_parser = StructuredOutputParser.from_response_schemas(
            response_schemas
        )
        format_instructions = output_parser.get_format_instructions()
        system_prompt = f"""
        You are an advanced static analyzer for programs and source codes written in Java. Your duty is to detect resource leakage bugs in the given Java codes delimited by ####. Each of this Java codes contain a method implementation, and you are supposed to analyze the he body this method. For example, if the given Java code is like: "final static void FooBar(){{...}}", you only look and analyze the issues surrounded by curly braces and the codes inside of them.

        REMEMBER: Only report the resources that are acquired and not released afterwards. The general pattern is enclosed in %%%% in the following line:
        %%%%
        Allocate resource
        try {{
            do some stuff
        }} finally {{
            close resource
        }}
        %%%%

        REMEMBER: Do not report the resources that uses Java 7's try-with-resources syntax. The pattern for this case is enclosed in &&&& in the following line:
        &&&&
            try (allocate resources){{
                do some stuff
            }}
        &&&&

        Please note that in this syntax, the resource will be released and closed automatically after the try block finishes. Hence, it does not require explicitly to release the allocated resource. In this syntax, the allocation part will be inside of the paranthesis in front of `try` keyword.


        REMEMBER: Only report the cases that you are sure and confident that they are high likely a resource leakage potential bug. Do not report if you are not sure.

        Use the following formatting guideline for the output:

        << FORMATTING >>
        {format_instructions}
        """
        warning_inference_prompt = (
            lambda warning: f"""
            you are given a warning message produced by an static analyzer and it is delimited by @@@@. The template of this message is as follows: "resource of type [RESOURCE_TYPE] acquired by call to [CALL_INVOKATION] at line [NUM] is not released after line [NUM].
            
            @@@@
            {warning}
            @@@@
            
            You need to analyze this warning and show the output in the provided output format:
            The output format should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

            ```json
            {{
                "targets": ["RESOURCE_TYPE", "CALL_INVOKATION"],
                "result": "True",
                "explanation": "resource of type [RESOURCE_TYPE] acquired by call to [CALL_INVOKATION] is not released afterwards"
            }}
            ```
            
            As you can see, you need to remove the line numbers and [NUM] from the explanation of warning.
            
            """
        )
    # end prompt templates 

    #cache pair example (bug type specific) 
    pair_examples_list = {"one-shot": [0]*number_of_folds, "few-shot":[0]*number_of_folds}
    if os.path.exists(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/pair_examples_list.pkl"):
        print(f"loaded /home/hhjww/Downloads/effectivenessPackage/src/caching/pair_examples_list.pkl")
        with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/pair_examples_list.pkl", 'rb') as f:
                pair_examples_list = pickle.load(f)
    else:
        
        for i in range (2):
            if (i == 0): 
                strategy = "one-shot"
                num_of_pair_ex= 1
            if (i == 1): 
                strategy = "few-shot"
                num_of_pair_ex = 3
            for fold in range(number_of_folds):
                buggy_folds = folds[f"{bug_type_master}_buggy"]
                clean_folds = folds[f"{bug_type_master}_clean"]
                validation_data = validation_data_list[fold]

                for index, validation_example in enumerate(validation_data):
                    examples = []
                    buggy_examples = []
                    clean_examples = []

                    # buggy_val_data = buggy_folds[fold_index]["val"]
                    # clean_val_data = clean_folds[fold_index]["val"]
                    #print(buggy_folds[index]["train"])

                    buggy_train_data = buggy_folds[fold]["train"]
                    clean_train_data = clean_folds[fold]["train"]

                    buggy_samples = buggy_train_data.sample(n=num_of_pair_ex)
                    clean_samples = clean_train_data.sample(n=num_of_pair_ex)

                    for index, val in buggy_samples.iterrows():
                        snippet = val["snippet"]
                        warning = val["warning"]
                        target = val["buggy"]

                        # entry = (snippet, warning, target)
                        entry = {"snippet": snippet, "warning": warning, "target": target}
                        buggy_examples.append(entry)

                    for index, val in clean_samples.iterrows():
                        snippet = val["snippet"]
                        warning = val["warning"]
                        target = val["buggy"]

                        # entry = (snippet, warning, target)
                        entry = {"snippet": snippet, "warning": warning, "target": target}
                        clean_examples.append(entry)

                    examples = list(zip(buggy_examples, clean_examples))
                    pair_examples_list[strategy][fold] = examples
                #end for
            #end for
        #end for
        # cache created pair example
        with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/pair_examples_list.pkl", 'wb') as f:
            pickle.dump(pair_examples_list, f)
            print("cached /home/hhjww/Downloads/effectivenessPackage/src/caching/pair_examples_list.pkl")
    #end elif 
    # end caching paired exampels list 

    #caching example prompt 
    # general checking
    # check complete then partial
    example_progress = {
        "bug_type" : bug_type_master,
        "strategy" : strategy,
        "fold" : 0,
        "index" : 0 
        }
    need_continue_fold = False
    need_continue_index = False
    list_examples = []
    if os.path.exists(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_examples_complete.pkl"):
        print(f"found /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_examples_complete.pkl")
        return
    
    elif os.path.exists(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_examples_part.pkl"):
        print(f"continuing /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_examples_part.pkl")
        with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_examples_part.pkl", 'rb') as f:
            list_examples = pickle.load(f)
        # now check for existing example_progress
        with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_example_progress.json", "r") as f:
            example_progress = json.load(f)
            print(example_progress)
        #set this flag to true if need to continue from previous recorded spot
        need_continue_fold = True
        need_continue_index = True
        #after setting value, set this value to false


    # few-shot 
    if (strategy == "few-shot"):
        list_examples = [0]*number_of_folds
        fold = 0
        #for fold in range(number_of_folds):
        while fold < number_of_folds: 
            if (need_continue_fold):
                fold = example_progress["fold"]
                need_continue_fold = False
            #end continuation check
            example_progress["fold"] = fold
            model1_output_data = {
                "fold": [],
                "snippet": [],
                "warning": [],
                "buggy": [],
                "predicted_buggy": [],
                "predicted_targets": [],
                "predicted_explanation": [],
            }
            model2_output_data = {
                "fold": [],
                "snippet": [],
                "warning": [],
                "buggy": [],
                "predicted_buggy": [],
                "predicted_targets": [],
                "predicted_explanation": [],
            }

            buggy_folds = folds[f"{bug_type_master}_buggy"]
            clean_folds = folds[f"{bug_type_master}_clean"]

            print(f"\t\tFOLD: {fold}")
            #validation_data = get_validation_data(
            #    fold, buggy_folds, clean_folds
            #)
            validation_data = validation_data_list[fold]
            list_examples[fold] = [0]*len(validation_data)
            #for index, validation_example in enumerate(validation_data):
            index = 0
            while index < len(validation_data):
                if (need_continue_index):
                    index = example_progress["index"]
                    need_continue_index = False
                #end of continuation check
                example_progress["index"] = index
                #example_pairs = get_pair_examples(
                #    fold, buggy_folds, clean_folds, 3
                #)
                example_pairs = pair_examples_list[strategy][fold]

                if bug_type_master == "NULL_DEREFERENCE":
                    # Buggy Examples 0
                    example0_buggy_code = example_pairs[0][0]["snippet"]
                    example0_buggy_warning = example_pairs[0][0]["warning"]
                    example0_buggy_response = inference_model.predict(
                       warning_inference_prompt(example0_buggy_warning)
                    )
                    #example0_buggy_response = warning_inference_prompt(example0_buggy_warning)

                    # Clean Examples 0
                    example0_clean_code = example_pairs[0][1]["snippet"]
                    example0_clean_warning = "This code is clean, and it has no null dereference potential bugs."
                    example0_clean_response = """
                        ```json
                        {
                            "targets": [],
                            "result": "False",
                            "explanation": "This code is clean, and it has no null dereference potential bugs."
                        }
                        ```
                    """
                    # Buggy Examples 1
                    example1_buggy_code = example_pairs[1][0]["snippet"]
                    example1_buggy_warning = example_pairs[1][0]["warning"]
                    example1_buggy_response = inference_model.predict(
                       warning_inference_prompt(example1_buggy_warning)
                    )
                    #example1_buggy_response = warning_inference_prompt(example1_buggy_warning)

                    # Clean Examples 1
                    example1_clean_code = example_pairs[1][1]["snippet"]
                    example1_clean_warning = "This code is clean, and it has no null dereference potential bugs."
                    example1_clean_response = """
                        ```json
                        {
                            "targets": [],
                            "result": "False",
                            "explanation": "This code is clean, and it has no null dereference potential bugs."
                        }
                        ```
                    """
                    # Buggy Examples 2
                    example2_buggy_code = example_pairs[2][0]["snippet"]
                    example2_buggy_warning = example_pairs[2][0]["warning"]
                    example2_buggy_response = inference_model.predict(
                       warning_inference_prompt(example2_buggy_warning)
                    )
                    #example2_buggy_response = warning_inference_prompt(example2_buggy_warning)

                    # Clean Examples 2
                    example2_clean_code = example_pairs[2][1]["snippet"]
                    example2_clean_warning = "This code is clean, and it has no null dereference potential bugs."
                    example2_clean_response = """
                        ```json
                        {
                            "targets": [],
                            "result": "False",
                            "explanation": "This code is clean, and it has no null dereference potential bugs."
                        }
                        ```
                    """
                elif bug_type_master == "RESOURCE_LEAK":
                    # Buggy Examples 0
                    example0_buggy_code = example_pairs[0][0]["snippet"]
                    example0_buggy_warning = example_pairs[0][0]["warning"]
                    example0_buggy_response = inference_model.predict(
                       warning_inference_prompt(example0_buggy_warning)
                    )
                    #example0_buggy_response = warning_inference_prompt(example0_buggy_warning)

                    # Clean Examples 0
                    example0_clean_code = example_pairs[0][1]["snippet"]
                    example0_clean_warning = "This code is clean, and it has no potential resource leakage bugs."
                    example0_clean_response = """
                        ```json
                        {
                            "targets": [],
                            "result": "False",
                            "explanation": "This code is clean, and it has no potential resource leakage bugs."
                        }
                        ```
                    """
                    # Buggy Examples 1
                    example1_buggy_code = example_pairs[1][0]["snippet"]
                    example1_buggy_warning = example_pairs[1][0]["warning"]
                    example1_buggy_response = inference_model.predict(
                       warning_inference_prompt(example1_buggy_warning)
                    )
                    #example1_buggy_response = warning_inference_prompt(example1_buggy_warning)

                    # Clean Examples 1
                    example1_clean_code = example_pairs[1][1]["snippet"]
                    example1_clean_warning = "This code is clean, and it has no potential resource leakage bugs."
                    example1_clean_response = """
                        ```json
                        {
                            "targets": [],
                            "result": "False",
                            "explanation": "This code is clean, and it has no potential resource leakage bugs."
                        }
                        ```
                    """
                    # Buggy Examples 2
                    example2_buggy_code = example_pairs[2][0]["snippet"]
                    example2_buggy_warning = example_pairs[2][0]["warning"]
                    example2_buggy_response = inference_model.predict(
                       warning_inference_prompt(example2_buggy_warning)
                    )
                    #example2_buggy_response = warning_inference_prompt(example2_buggy_warning)

                    # Clean Examples 2
                    example2_clean_code = example_pairs[2][1]["snippet"]
                    example2_clean_warning = "This code is clean, and it has no potential resource leakage bugs."
                    example2_clean_response = """
                        ```json
                        {
                            "targets": [],
                            "result": "False",
                            "explanation": "This code is clean, and it has no potential resource leakage bugs."
                        }
                        ```
                    """
                #end of prompting for examples, creating example list
                examples = [
                    {
                        "input": f"####{example0_buggy_code}####",
                        "output": example0_buggy_response,
                    },
                    {
                        "input": f"####{example0_clean_code}####",
                        "output": example0_clean_response,
                    },
                    {
                        "input": f"####{example1_buggy_code}####",
                        "output": example1_buggy_response,
                    },
                    {
                        "input": f"####{example1_clean_code}####",
                        "output": example1_clean_response,
                    },
                    {
                        "input": f"####{example2_buggy_code}####",
                        "output": example2_buggy_response,
                    },
                    {
                        "input": f"####{example2_clean_code}####",
                        "output": example2_clean_response,
                    },
                ]

                list_examples[fold][index] = examples
                print(f"completed example {fold} - {index}") 
                # after each completed example, save progress
                with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_examples_part.pkl", 'wb') as f:
                    pickle.dump(list_examples, f)
                    print(f"cached /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_examples_part.pkl")
                # save progress json  
                with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_example_progress.json", 'w') as f:
                    print(type(example_progress))
                    json.dump(example_progress, f)
                    print(f"progress logged /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_example_progress.json")
                # end of caching and logging                 
                index += 1
            #end for index 

            #debug
            #if fold > 0:
            #    break
            #debug
            fold += 1
        #end for fold

        # if fold finishes, means the whole list of examples are completed. write the examples_complete file. 
        with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_examples_complete.pkl", 'wb') as f:
            pickle.dump(list_examples, f)
            print(f"cached /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_examples_complete.pkl")
            print("examples caching completed")
        # example list: [folds][validation dataset]       
    
    # one-shot 
    if (strategy == "one-shot"):
        list_examples = [0]*number_of_folds
        fold = 0
        #for fold in range(number_of_folds):
        while fold < number_of_folds: 
            if (need_continue_fold):
                fold = example_progress["fold"]
                need_continue_fold = False
            #end continuation check
            example_progress["fold"] = fold
            model1_output_data = {
                "fold": [],
                "snippet": [],
                "warning": [],
                "buggy": [],
                "predicted_buggy": [],
                "predicted_targets": [],
                "predicted_explanation": [],
            }
            model2_output_data = {
                "fold": [],
                "snippet": [],
                "warning": [],
                "buggy": [],
                "predicted_buggy": [],
                "predicted_targets": [],
                "predicted_explanation": [],
            }

            buggy_folds = folds[f"{bug_type_master}_buggy"]
            clean_folds = folds[f"{bug_type_master}_clean"]

            print(f"\t\tFOLD: {fold}")
            #validation_data = get_validation_data(
            #    fold, buggy_folds, clean_folds
            #)
            validation_data = validation_data_list[fold]
            list_examples[fold] = [0]*len(validation_data)
            #for index, validation_example in enumerate(validation_data):
            index = 0
            while index < len(validation_data):
                if (need_continue_index):
                    index = example_progress["index"]
                    need_continue_index = False
                #end of continuation check
                example_progress["index"] = index
                #example_pairs = get_pair_examples(
                #    fold, buggy_folds, clean_folds, 3
                #)
                example_pairs = pair_examples_list[strategy][fold]

                if bug_type_master == "NULL_DEREFERENCE":
                    # Buggy Examples
                    example0_buggy_code = example_pairs[0][0]["snippet"]
                    example0_buggy_warning = example_pairs[0][0]["warning"]
                    example0_buggy_response = """
                        ```json
                        {
                            "result": "True",
                            "explanation": "The warning is correct. The reported case is a potential null dereference bug."
                        }
                        ```
                    """

                    # Clean Examples
                    example0_clean_code = example_pairs[0][1]["snippet"]
                    example0_clean_warning = example_pairs[0][1]["warning"]
                    example0_clean_response = """
                        ```json
                        {
                            "result": "False",
                            "explanation": "The warning is a false positive and a wrong indicator of a potential null dereference bug."
                        }
                        ```
                    """
                elif bug_type == "RESOURCE_LEAK":
                    # Buggy Examples
                    example0_buggy_code = example_pairs[0][0]["snippet"]
                    example0_buggy_warning = example_pairs[0][0]["warning"]
                    example0_buggy_response = """
                        ```json
                        {
                            "result": "True",
                            "explanation": "The warning is correct. The reported case is a potential resource leakage bug."
                        }
                        ```
                    """

                    # Clean Examples
                    example0_clean_code = example_pairs[0][1]["snippet"]
                    example0_clean_warning = example_pairs[0][1]["warning"]
                    example0_clean_response = """
                        ```json
                        {
                            "result": "False",
                            "explanation": "The warning is a false positive and a wrong indicator of a potential resource leakage bug."
                        }
                        ```
                    """

                examples = [
                    {
                        "input": f"####{example0_buggy_code}####",
                        "output": example0_buggy_response,
                    },
                    {
                        "input": f"####{example0_clean_code}####",
                        "output": example0_clean_response,
                    },
                ]

                list_examples[fold][index] = examples
                print(f"completed example {fold} - {index}") 
                # after each completed example, save progress
                with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_examples_part.pkl", 'wb') as f:
                    pickle.dump(list_examples, f)
                    print(f"cached /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_examples_part.pkl")
                # save progress json  
                with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_example_progress.json", 'w') as f:
                    print(type(example_progress))
                    json.dump(example_progress, f)
                    print(f"progress logged /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_example_progress.json")
                # end of caching and logging                 
                index += 1
            #end for index 

            #debug
            #if fold > 0:
            #    break
            #debug
            fold += 1
        #end for fold

        # if fold finishes, means the whole list of examples are completed. write the examples_complete file. 
        with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_examples_complete.pkl", 'wb') as f:
            pickle.dump(list_examples, f)
            print(f"cached /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type_master}_{strategy}_examples_complete.pkl")
            print("examples caching completed")
        # example list: [folds][validation dataset]       
                    



    return

<>:113: SyntaxWarning: invalid escape sequence '\.'
<>:113: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_36087/1260820243.py:113: SyntaxWarning: invalid escape sequence '\.'
  description='Any explanation you have for your decision. Explain your answer briefly. Make sure the text in the explanation escape double quotes with \. For example: "example"',


In [8]:
#cache_data("NULL_DEREFERENCE", "zero-shot")
#cache_data("NULL_DEREFERENCE", "one-shot")
#cache_data("NULL_DEREFERENCE", "few-shot")
#cache_data("RESOURCE_LEAK", "zero-shot")
#cache_data("RESOURCE_LEAK", "one-shot")
cache_data("RESOURCE_LEAK", "few-shot")

found /home/hhjww/Downloads/effectivenessPackage/src/caching/folds.pkl

loaded /home/hhjww/Downloads/effectivenessPackage/src/caching/validation_data_list.pkl

loaded /home/hhjww/Downloads/effectivenessPackage/src/caching/pair_examples_list.pkl

found /home/hhjww/Downloads/effectivenessPackage/src/caching/RESOURCE_LEAK_few-shot_examples_complete.pkl

Ok so in the previous section, we cached examples and all required files and made it all determinstic at least in between runs
In the next section we will implement something similar for running the experiment. 

this function cleans data.
we implement custom json cleaning because deepseek/qwen make very unclean output.

In [9]:
# so i have been loading data in a few different ways
# sometimes i use a parser, sometimes i use json load,  i think i should just do json load for all.
def custom_clean_json(text):
    #print(f"before{text}")
    pattern = r'json.*?\{(?:[^{}]*|(?:\{[^{}]*\}))*\}'
    match = re.search(pattern, text, re.DOTALL)
    if match:
        text = match.group()
        #print("json matched")
        pattern = r'json|\\n|\n|\\t|\t'
        text = re.sub(pattern, '', text)
        text = re.sub(r'\bTrue\b|\bFalse\b', lambda m: m.group().lower(), text)
        text = re.sub(r',\s*([}\]])', r'\1', text)
    #print(f"after{text}")
    return text

the following code raises a time out exception manually, may not work on ur system. 
this is because ollama hangs at basically random and requiring restarts 


In [10]:
class OllamaTimeout(Exception):
    pass

def handler(signum, frame):
    raise OllamaTimeout("Ollama is stuck")

signal.signal(signal.SIGALRM, handler)



<Handlers.SIG_DFL: 0>

now we actually run the experiments 

In [11]:
#prepare RQ 1 
experiment = "RQ1"
#bug_type = "NULL_DEREFERENCE" #change this to run the other one!
bug_type = "RESOURCE_LEAK"
strategy = "few-shot"
#strategy = "one-shot"
#strategy = "zero-shot"


folds = {}
with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/folds.pkl", 'rb') as f:
    folds = pickle.load(f)

validation_data_list = []
with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/validation_data_list.pkl", 'rb') as f:
    validation_data_list = pickle.load(f)

pair_examples_list = {}
with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/pair_examples_list.pkl", 'rb') as f:
    pair_examples_list = pickle.load(f)

list_examples = []
with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_examples_complete.pkl", 'rb') as f:
    list_examples = pickle.load(f)

the next cell loads the schemas and prompts 

In [12]:
if bug_type == "NULL_DEREFERENCE":
    targets = ResponseSchema(
        name="targets",
        description="Give me the list of all potential objects, method calls, and function calls that are suspected to be the cases of null dereference bugs detected by the analysis. Make sure the given potential suspects are the ones that their values are not being checked for being null or not in the code. The list should be in a format like this: [a, b, c, d]. If there weren't any potential suspects, you should return an empty list like this: [], which means that there is no cases of null dereference bug",
    )
    results = ResponseSchema(
        name="result",
        description="What is the result of the analysis? Is the given Java code buggy with a potential null dereference bug? Answer True if yes, Answer False if no.",
    )
    explanation = ResponseSchema(
        name="explanation",
        description='Any explanation you have for your decision. Explain your answer briefly. Make sure the text in the explanation escape double quotes with \. For example: "example"',
    )
    response_schemas = [targets, results, explanation]
    output_parser = StructuredOutputParser.from_response_schemas(
        response_schemas
    )
    format_instructions = output_parser.get_format_instructions()
    system_prompt = f"""
        You are an advanced static analyzer for programs and source codes written in Java. Your duty is to detect null dereference bugs in the given Java codes delimited by ####. Each of this Java codes contain a method implementation, and you are supposed to analyze the he body this method. For example, if the given Java code is like: "final static void FooBar(){{...}}", you only look and analyze the issues surrounded by curly braces and the codes inside of them.

        REMEMBER: Please make sure that if any objects and method or function calls and invokations could potentially return a null value and become a potential null dereference bug, please report them and consider the given Java code as a buggy code with null dereference issue. 

        REMEMBER: If the value of the objects or the result of the method and function calls and invokations are being checked with if statements for being null or not, this means that the null dereference bug is handled, and this is not a case of null dereference bug, and those objects and method or function calls and invokations are not buggy.

        REMEMBER: Only report the objects or method and function calls and invokations that you are sure and confident that they are high likely a null dereference potential bug. Do not report if you are not sure.

        Use the following formatting guideline for the output:

        << FORMATTING >>
        {format_instructions}
    """
    warning_inference_prompt = (
        lambda warning: f"""
        Which object or method call or function call could be null null in the following warning message delimited by @@@@?
        
        @@@@
        {warning}
        @@@@
        
        Please use the following output format for reporting the object:
        The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

        ```json
        {{
            "targets": ["OBJECT_NAME"],
            "result": "True",
            "explanation": "The object OBJECT_NAME could potentially be null and dereferenced later"
        }}
        ```
        
        REMEMBER: In case instead of an object, a method call or function call was the suspect, you need to follow the following output format:
        
        ```json
        {{
            "targets": ["METHOD_INVOKATION"],
            "result": "True",
            "explanation": "The call to METHOD_INVOKATION could potentially return null and dereferenced later"
        }}
        ```
        
        """
    )

elif bug_type == "RESOURCE_LEAK":
    targets = ResponseSchema(
        name="targets",
        description="Give me the list of all potential resources and allocations that are suspected to be the cases of resource leakage bugs detected by the analysis. Make sure the detected potential suspects are the resources that are acquired and not released afterward by the end of the method. The list should be in a format like this: [a, b, c, d]. If there weren't any potential suspects, you should return an empty list like this: [], which means that there is no cases of resource leakage bug",
    )
    result = ResponseSchema(
        name="result",
        description="What is the result of the analysis? Is the given Java code buggy with a resource leakage bug? Answer True if yes, Answer False if no.",
    )
    explanation = ResponseSchema(
        name="explanation",
        description="Any explanation you have for your decision. Explain your answer briefly. Make sure the text in the explanation is JSON parsable or JSON friendly",
    )
    response_schemas = [targets, result, explanation]
    output_parser = StructuredOutputParser.from_response_schemas(
        response_schemas
    )
    format_instructions = output_parser.get_format_instructions()
    system_prompt = f"""
    You are an advanced static analyzer for programs and source codes written in Java. Your duty is to detect resource leakage bugs in the given Java codes delimited by ####. Each of this Java codes contain a method implementation, and you are supposed to analyze the he body this method. For example, if the given Java code is like: "final static void FooBar(){{...}}", you only look and analyze the issues surrounded by curly braces and the codes inside of them.

    REMEMBER: Only report the resources that are acquired and not released afterwards. The general pattern is enclosed in %%%% in the following line:
    %%%%
    Allocate resource
    try {{
        do some stuff
    }} finally {{
        close resource
    }}
    %%%%

    REMEMBER: Do not report the resources that uses Java 7's try-with-resources syntax. The pattern for this case is enclosed in &&&& in the following line:
    &&&&
        try (allocate resources){{
            do some stuff
        }}
    &&&&

    Please note that in this syntax, the resource will be released and closed automatically after the try block finishes. Hence, it does not require explicitly to release the allocated resource. In this syntax, the allocation part will be inside of the paranthesis in front of `try` keyword.


    REMEMBER: Only report the cases that you are sure and confident that they are high likely a resource leakage potential bug. Do not report if you are not sure.

    Use the following formatting guideline for the output:

    << FORMATTING >>
    {format_instructions}
    """
    warning_inference_prompt = (
        lambda warning: f"""
        you are given a warning message produced by an static analyzer and it is delimited by @@@@. The template of this message is as follows: "resource of type [RESOURCE_TYPE] acquired by call to [CALL_INVOKATION] at line [NUM] is not released after line [NUM].
        
        @@@@
        {warning}
        @@@@
        
        You need to analyze this warning and show the output in the provided output format:
        The output format should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

        ```json
        {{
            "targets": ["RESOURCE_TYPE", "CALL_INVOKATION"],
            "result": "True",
            "explanation": "resource of type [RESOURCE_TYPE] acquired by call to [CALL_INVOKATION] is not released afterwards"
        }}
        ```
        
        As you can see, you need to remove the line numbers and [NUM] from the explanation of warning.
        
        """
    )

<>:12: SyntaxWarning: invalid escape sequence '\.'
<>:12: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_36087/1368190677.py:12: SyntaxWarning: invalid escape sequence '\.'
  description='Any explanation you have for your decision. Explain your answer briefly. Make sure the text in the explanation escape double quotes with \. For example: "example"',


the next cell loads the incomplete saves if available, the same logic as caching
This cell is unused as this is merged with the rest of the code in the next cell

In [13]:
#caching experiment prompt 
# general checking
# check complete then partial
experiment_progress = {
    "bug_type" : bug_type,
    "strategy" : strategy,
    "fold" : 0,
    "index" : 0 
    }
need_continue_fold = False
need_continue_index = False

list_results = {"list_model1_output_data" : [0]*number_of_folds, "list_model2_output_data" : [0]*number_of_folds} #list_results = {"list_model1_output_data":[folds], "list_model2_output_data":[folds]}, if available
if os.path.exists(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_complete.pkl"):
    print(f"found /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_complete.pkl")
    print("stopping experiments")
    raise RuntimeError("Forced crash")
    

elif os.path.exists(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl"):
    print(f"continuing /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl")
    with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl", 'rb') as f:
        list_results = pickle.load(f)
    # now check for existing example_progress
    with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_progress.json", "r") as f:
        experiment_progress = json.load(f)
        print(experiment_progress)
    #set this flag to true if need to continue from previous recorded spot
    need_continue_fold = True
    need_continue_index = True
    #after setting value, set this value to false

Few shot experiment 

In [ ]:
complete_flag = False
while not complete_flag: 
    try:    
        #set timeout to s
        signal.alarm(200) 
        #caching experiment result 
        # general checking
        # check complete then partial    
        experiment_progress = {
            "bug_type" : bug_type,
            "strategy" : strategy,
            "fold" : 0,
            "index" : 0 
            }
        need_continue_fold = False
        need_continue_index = False

        list_results = {"list_model1_output_data" : [0]*number_of_folds, "list_model2_output_data" : [0]*number_of_folds} #list_results = {"list_model1_output_data":[folds], "list_model2_output_data":[folds]}, if available
        if os.path.exists(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_complete.pkl"):
            print(f"found /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_complete.pkl")
            print("stopping experiments")
            raise RuntimeError("Forced crash")
            

        elif os.path.exists(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl"):
            print(f"continuing /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl")
            with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl", 'rb') as f:
                list_results = pickle.load(f)
            # now check for existing example_progress
            with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_progress.json", "r") as f:
                experiment_progress = json.load(f)
                print(experiment_progress)
            #set this flag to true if need to continue from previous recorded spot
            need_continue_fold = True
            need_continue_index = True
            #after setting value, set this value to false

            
        # Few-Shot Strategy
        print(f"\tStrategy: {strategy}")
        # TODO: replace this with while loop for counter 
        fold = 0
        #for fold in range(number_of_folds):
        while fold < number_of_folds: 
            if (need_continue_fold):
                fold = experiment_progress["fold"]
                need_continue_fold = False
            #end continuation check
            experiment_progress["fold"] = fold
            
            model1_output_data = {
                "fold": [],
                "snippet": [],
                "warning": [],
                "buggy": [],
                "predicted_buggy": [],
                "predicted_targets": [],
                "predicted_explanation": [],
            }
            model2_output_data = {
                "fold": [],
                "snippet": [],
                "warning": [],
                "buggy": [],
                "predicted_buggy": [],
                "predicted_targets": [],
                "predicted_explanation": [],
            }

            list_results["list_model1_output_data"][fold] = model1_output_data
            list_results["list_model2_output_data"][fold] = model2_output_data


            buggy_folds = folds[f"{bug_type}_buggy"]
            clean_folds = folds[f"{bug_type}_clean"]

            #print(f"\t\tFOLD: {fold}")
            validation_data = validation_data_list[fold]
            
            index = 0
            #for index, validation_example in enumerate(validation_data):
            while index < len(validation_data):
                if (need_continue_index):
                    index = experiment_progress["index"]
                    need_continue_index = False
                #end continuation check
                experiment_progress["index"] = index
                # example_pairs = self.get_pair_examples(
                #     fold, buggy_folds, clean_folds, 3
                # )
                validation_example = validation_data[index]

                #reminder: pair_examples_list = {"one-shot": [0]*number_of_folds, "few-shot":[0]*number_of_folds}
                example_pairs = pair_examples_list[strategy][fold]
                
                # examples = [
                #     {
                #         "input": f"####{example0_buggy_code}####",
                #         "output": example0_buggy_response,
                #     },
                #     {
                #         "input": f"####{example0_clean_code}####",
                #         "output": example0_clean_response,
                #     },
                #     {
                #         "input": f"####{example1_buggy_code}####",
                #         "output": example1_buggy_response,
                #     },
                #     {
                #         "input": f"####{example1_clean_code}####",
                #         "output": example1_clean_response,
                #     },
                #     {
                #         "input": f"####{example2_buggy_code}####",
                #         "output": example2_buggy_response,
                #     },
                #     {
                #         "input": f"####{example2_clean_code}####",
                #         "output": example2_clean_response,
                #     },
                # ] 
                #reminder: above is the output that is saved for each fold and index.
                examples = list_examples[fold][index] 



                example_prompt = ChatPromptTemplate.from_messages(
                    [
                        ("human", "{input}"),
                        ("ai", "{output}"),
                    ]
                )

                few_shot_prompt = FewShotChatMessagePromptTemplate(
                    example_prompt=example_prompt,
                    examples=examples,
                )

                prompt_template = ChatPromptTemplate.from_messages(
                    [
                        ("system", "{input_system_prompt}"),
                        few_shot_prompt,
                        ("human", "####{input_code}####"),
                    ]
                )
                #print(f"\t\t\tValidation Example Index: {index}")
                validation_example_snippet = validation_example["snippet"]
                validation_example_warning = validation_example["warning"]
                validation_example_target = validation_example["target"]

                #llm_model1 = self.chatgpt4
                #llm_model2 = self.chatgpt3
                llm_model1 = deepseek1b
                llm_model2 = deepseek1b

                #chain_model1 = LLMChain(llm=llm_model1, prompt=prompt_template) changed this cuz this is old
                chain_model1 = prompt_template | llm_model1 | StrOutputParser()
                response_model1 = chain_model1.invoke(
                    {
                        "input_system_prompt": system_prompt,
                        "input_code": validation_example_snippet,
                    }
                )
                #response_model1 = 'dkalwjdnasdnwakljdnawl json{"result":"dummy1", "targets":[], "explanation":"dummy1explanation"}'

                try:
                    # output_dict_model1 = output_parser.parse(
                    #     response_model1["text"]
                    # )
                    text = response_model1
                    text = custom_clean_json(text) 
                    output_dict_model1 = json.loads(text)
                    #this replaces original parser
                    predicted_result1 = output_dict_model1["result"]
                    predicted_targets1 = output_dict_model1["targets"]
                    predicted_explanation1 = output_dict_model1["explanation"]
                except Exception as e:
                    #print(e)
                    #print(f"\t\t\tJSON Parse Error")
                    predicted_result1 = "fail"
                    predicted_targets1 = "fail"
                    predicted_explanation1 = response_model1

                # adding the results to the model output dictionary for producing the results CSV
                #print(f"\t\t\t{llm_model1.model} has processed the example {index}")

                list_results["list_model1_output_data"][fold]["fold"].append(fold)
                list_results["list_model1_output_data"][fold]["snippet"].append(validation_example_snippet)
                list_results["list_model1_output_data"][fold]["warning"].append(validation_example_warning)
                list_results["list_model1_output_data"][fold]["buggy"].append(validation_example_target)
                list_results["list_model1_output_data"][fold]["predicted_buggy"].append(predicted_result1)
                list_results["list_model1_output_data"][fold]["predicted_targets"].append(predicted_targets1)
                list_results["list_model1_output_data"][fold]["predicted_explanation"].append(
                    predicted_explanation1
                )

                #changed this line also cuz previously was old
                chain_model2 = prompt_template | llm_model2 | StrOutputParser()
                response_model2 = chain_model2.invoke(
                    {
                        "input_system_prompt": system_prompt,
                        "input_code": validation_example_snippet,
                    }
                )

                #response_model2 = 'dkalwjdnasdnwakljdnawl json{"result":"dummy2", "targets":[], "explanation":"dummy2explanation"}'

                try:
                    # output_dict_model2 = output_parser.parse(
                    #     response_model2["text"]
                    # )
                    text = response_model2
                    text = custom_clean_json(text) 
                    output_dict_model2 = json.loads(text)
                    #this replaces original parser
                    predicted_result2 = output_dict_model2["result"]
                    predicted_targets2 = output_dict_model2["targets"]
                    predicted_explanation2 = output_dict_model2["explanation"]
                except Exception as e:
                    #print(e)
                    #print(text)
                    #print(f"\t\t\t>> JSON Parse Error")
                    predicted_result2 = "fail"
                    predicted_targets2 = "fail"
                    predicted_explanation2 = response_model2

                # adding the results to the model output dictionary for producing the results CSV
                #print(f"\t\t\t{llm_model2.model} has processed the example {index}")

                list_results["list_model2_output_data"][fold]["fold"].append(fold)
                list_results["list_model2_output_data"][fold]["snippet"].append(validation_example_snippet)
                list_results["list_model2_output_data"][fold]["warning"].append(validation_example_warning)
                list_results["list_model2_output_data"][fold]["buggy"].append(validation_example_target)
                list_results["list_model2_output_data"][fold]["predicted_buggy"].append(predicted_result2)
                list_results["list_model2_output_data"][fold]["predicted_targets"].append(predicted_targets2)
                list_results["list_model2_output_data"][fold]["predicted_explanation"].append(
                    predicted_explanation2
                )

                #print(f"\t\t\t----------------------------------------------")
                
                print(f"completed experiment {fold} - {index}") 

                # after each completed example, save progress
                # note that this only runs once both models completes. avoiding issue where one is complete and the other get stuck 
                with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl", 'wb') as f:
                    pickle.dump(list_results, f)
                    #print(f"cached /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl")
                # save progress json  
                with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_progress.json", 'w') as f:
                    #print(experiment_progress)
                    json.dump(experiment_progress, f)
                    #print(f"progress logged /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_progress.json")
                # end of caching and logging  

                #time.sleep(1)
                index += 1
            #end while index
            
            save_results(
                pd.DataFrame(list_results["list_model1_output_data"][fold]),
                experiment,
                bug_type,
                strategy,
                llm_model1.model,
                fold,
            )

            save_results(
                pd.DataFrame(list_results["list_model2_output_data"][fold]),
                experiment,
                bug_type,
                strategy,
                llm_model2.model,
                fold,
            )
            fold += 1
        #end while fold 
        # if fold finishes, means the whole list of examples are completed. write the examples_complete file. 
        with open(f"/home/hhjww/Downloads/effectivenessPackage/src/results/{bug_type}_{strategy}_experiment_complete.pkl", 'wb') as f:
            pickle.dump(list_results, f)
            print(f"cached /home/hhjww/Downloads/effectivenessPackage/src/results/{bug_type}_{strategy}_experiment_complete.pkl")
            print("all result cache completed")
            complete_flag = True
    except OllamaTimeout as e:
        print("ollama timed out restarting")


Strategy: few-shot

completed experiment 0 - 0

zero shot testing 

In [ ]:
#prepare RQ 1 
experiment = "RQ1"
bug_type = "NULL_DEREFERENCE" #change this to run the other one!
#bug_type = "RESOURCE_LEAK"
#strategy = "few-shot"
#strategy = "one-shot"
strategy = "zero-shot"


folds = {}
with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/folds.pkl", 'rb') as f:
    folds = pickle.load(f)

validation_data_list = []
with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/validation_data_list.pkl", 'rb') as f:
    validation_data_list = pickle.load(f)

pair_examples_list = {}
with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/pair_examples_list.pkl", 'rb') as f:
    pair_examples_list = pickle.load(f)

list_examples = []
with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_examples_complete.pkl", 'rb') as f:
    list_examples = pickle.load(f)

In [ ]:
complete_flag = False
while not complete_flag: 
    try:    
        #set timeout to s
        signal.alarm(200) 
        #caching experiment result 
        # general checking
        # check complete then partial    
        experiment_progress = {
            "bug_type" : bug_type,
            "strategy" : strategy,
            "fold" : 0,
            "index" : 0 
            }
        need_continue_fold = False
        need_continue_index = False

        list_results = {"list_model1_output_data" : [0]*number_of_folds, "list_model2_output_data" : [0]*number_of_folds} #list_results = {"list_model1_output_data":[folds], "list_model2_output_data":[folds]}, if available
        if os.path.exists(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_complete.pkl"):
            print(f"found /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_complete.pkl")
            print("stopping experiments")
            raise RuntimeError("Forced crash")
            

        elif os.path.exists(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl"):
            print(f"continuing /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl")
            with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl", 'rb') as f:
                list_results = pickle.load(f)
            # now check for existing example_progress
            with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_progress.json", "r") as f:
                experiment_progress = json.load(f)
                print(experiment_progress)
            #set this flag to true if need to continue from previous recorded spot
            need_continue_fold = True
            need_continue_index = True
            #after setting value, set this value to false

            
        # Few-Shot Strategy
        print(f"\tStrategy: {strategy}")
        # TODO: replace this with while loop for counter 
        fold = 0
        #for fold in range(number_of_folds):
        while fold < number_of_folds: 
            if (need_continue_fold):
                fold = experiment_progress["fold"]
                need_continue_fold = False
            #end continuation check
            experiment_progress["fold"] = fold
            
            model1_output_data = {
                "fold": [],
                "snippet": [],
                "warning": [],
                "buggy": [],
                "predicted_buggy": [],
                "predicted_targets": [],
                "predicted_explanation": [],
            }
            model2_output_data = {
                "fold": [],
                "snippet": [],
                "warning": [],
                "buggy": [],
                "predicted_buggy": [],
                "predicted_targets": [],
                "predicted_explanation": [],
            }

            list_results["list_model1_output_data"][fold] = model1_output_data
            list_results["list_model2_output_data"][fold] = model2_output_data


            buggy_folds = folds[f"{bug_type}_buggy"]
            clean_folds = folds[f"{bug_type}_clean"]

            #print(f"\t\tFOLD: {fold}")
            validation_data = validation_data_list[fold]
            
            index = 0
            #for index, validation_example in enumerate(validation_data):
            while index < len(validation_data):
                if (need_continue_index):
                    index = experiment_progress["index"]
                    need_continue_index = False
                #end continuation check
                experiment_progress["index"] = index
                # example_pairs = self.get_pair_examples(
                #     fold, buggy_folds, clean_folds, 3
                # )
                validation_example = validation_data[index]

                prompt_template = ChatPromptTemplate.from_messages(
                    [
                        ("system", "{input_system_prompt}"),
                        ("human", "####{input_code}####"),
                    ]
                )

                #print(f"\t\t\tValidation Example Index: {index}")
                validation_example_snippet = validation_example["snippet"]
                validation_example_warning = validation_example["warning"]
                validation_example_target = validation_example["target"]

                #llm_model1 = self.chatgpt4
                #llm_model2 = self.chatgpt3
                llm_model1 = deepseek1b
                llm_model2 = deepseek1b

                #chain_model1 = LLMChain(llm=llm_model1, prompt=prompt_template) changed this cuz this is old
                chain_model1 = prompt_template | llm_model1 | StrOutputParser()
                response_model1 = chain_model1.invoke(
                    {
                        "input_system_prompt": system_prompt,
                        "input_code": validation_example_snippet,
                    }
                )
                #response_model1 = 'dkalwjdnasdnwakljdnawl json{"result":"dummy1", "targets":[], "explanation":"dummy1explanation"}'

                try:
                    # output_dict_model1 = output_parser.parse(
                    #     response_model1["text"]
                    # )
                    text = response_model1
                    text = custom_clean_json(text) 
                    output_dict_model1 = json.loads(text)
                    #this replaces original parser
                    predicted_result1 = output_dict_model1["result"]
                    predicted_targets1 = output_dict_model1["targets"]
                    predicted_explanation1 = output_dict_model1["explanation"]
                except Exception as e:
                    #print(e)
                    #print(f"\t\t\tJSON Parse Error")
                    predicted_result1 = "fail"
                    predicted_targets1 = "fail"
                    predicted_explanation1 = response_model1

                # adding the results to the model output dictionary for producing the results CSV
                #print(f"\t\t\t{llm_model1.model} has processed the example {index}")

                list_results["list_model1_output_data"][fold]["fold"].append(fold)
                list_results["list_model1_output_data"][fold]["snippet"].append(validation_example_snippet)
                list_results["list_model1_output_data"][fold]["warning"].append(validation_example_warning)
                list_results["list_model1_output_data"][fold]["buggy"].append(validation_example_target)
                list_results["list_model1_output_data"][fold]["predicted_buggy"].append(predicted_result1)
                list_results["list_model1_output_data"][fold]["predicted_targets"].append(predicted_targets1)
                list_results["list_model1_output_data"][fold]["predicted_explanation"].append(
                    predicted_explanation1
                )

                #changed this line also cuz previously was old
                chain_model2 = prompt_template | llm_model2 | StrOutputParser()
                response_model2 = chain_model2.invoke(
                    {
                        "input_system_prompt": system_prompt,
                        "input_code": validation_example_snippet,
                    }
                )

                #response_model2 = 'dkalwjdnasdnwakljdnawl json{"result":"dummy2", "targets":[], "explanation":"dummy2explanation"}'

                try:
                    # output_dict_model2 = output_parser.parse(
                    #     response_model2["text"]
                    # )
                    text = response_model2
                    text = custom_clean_json(text) 
                    output_dict_model2 = json.loads(text)
                    #this replaces original parser
                    predicted_result2 = output_dict_model2["result"]
                    predicted_targets2 = output_dict_model2["targets"]
                    predicted_explanation2 = output_dict_model2["explanation"]
                except Exception as e:
                    #print(e)
                    #print(text)
                    #print(f"\t\t\t>> JSON Parse Error")
                    predicted_result2 = "fail"
                    predicted_targets2 = "fail"
                    predicted_explanation2 = response_model2

                # adding the results to the model output dictionary for producing the results CSV
                #print(f"\t\t\t{llm_model2.model} has processed the example {index}")

                list_results["list_model2_output_data"][fold]["fold"].append(fold)
                list_results["list_model2_output_data"][fold]["snippet"].append(validation_example_snippet)
                list_results["list_model2_output_data"][fold]["warning"].append(validation_example_warning)
                list_results["list_model2_output_data"][fold]["buggy"].append(validation_example_target)
                list_results["list_model2_output_data"][fold]["predicted_buggy"].append(predicted_result2)
                list_results["list_model2_output_data"][fold]["predicted_targets"].append(predicted_targets2)
                list_results["list_model2_output_data"][fold]["predicted_explanation"].append(
                    predicted_explanation2
                )

                #print(f"\t\t\t----------------------------------------------")
                
                print(f"completed experiment {fold} - {index}") 

                # after each completed example, save progress
                # note that this only runs once both models completes. avoiding issue where one is complete and the other get stuck 
                with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl", 'wb') as f:
                    pickle.dump(list_results, f)
                    #print(f"cached /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl")
                # save progress json  
                with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_progress.json", 'w') as f:
                    #print(experiment_progress)
                    json.dump(experiment_progress, f)
                    #print(f"progress logged /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_progress.json")
                # end of caching and logging  

                #time.sleep(1)
                index += 1
            #end while index
            
            save_results(
                pd.DataFrame(list_results["list_model1_output_data"][fold]),
                experiment,
                bug_type,
                strategy,
                llm_model1.model,
                fold,
            )

            save_results(
                pd.DataFrame(list_results["list_model2_output_data"][fold]),
                experiment,
                bug_type,
                strategy,
                llm_model2.model,
                fold,
            )
            fold += 1
        #end while fold 
        # if fold finishes, means the whole list of examples are completed. write the examples_complete file. 
        with open(f"/home/hhjww/Downloads/effectivenessPackage/src/results/{bug_type}_{strategy}_experiment_complete.pkl", 'wb') as f:
            pickle.dump(list_results, f)
            print(f"cached /home/hhjww/Downloads/effectivenessPackage/src/results/{bug_type}_{strategy}_experiment_complete.pkl")
            print("all result cache completed")
            complete_flag = True
    except OllamaTimeout as e:
        print("ollama timed out restarting")


one shot experiment  

In [ ]:
complete_flag = False
while not complete_flag: 
    try:    
        #set timeout to s
        signal.alarm(200) 
        #caching experiment result 
        # general checking
        # check complete then partial    
        experiment_progress = {
            "bug_type" : bug_type,
            "strategy" : strategy,
            "fold" : 0,
            "index" : 0 
            }
        need_continue_fold = False
        need_continue_index = False

        list_results = {"list_model1_output_data" : [0]*number_of_folds, "list_model2_output_data" : [0]*number_of_folds} #list_results = {"list_model1_output_data":[folds], "list_model2_output_data":[folds]}, if available
        if os.path.exists(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_complete.pkl"):
            print(f"found /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_complete.pkl")
            print("stopping experiments")
            raise RuntimeError("Forced crash")
            

        elif os.path.exists(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl"):
            print(f"continuing /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl")
            with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl", 'rb') as f:
                list_results = pickle.load(f)
            # now check for existing example_progress
            with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_progress.json", "r") as f:
                experiment_progress = json.load(f)
                print(experiment_progress)
            #set this flag to true if need to continue from previous recorded spot
            need_continue_fold = True
            need_continue_index = True
            #after setting value, set this value to false

            
        # Few-Shot Strategy
        print(f"\tStrategy: {strategy}")
        # TODO: replace this with while loop for counter 
        fold = 0
        #for fold in range(number_of_folds):
        while fold < number_of_folds: 
            if (need_continue_fold):
                fold = experiment_progress["fold"]
                need_continue_fold = False
            #end continuation check
            experiment_progress["fold"] = fold
            
            model1_output_data = {
                "fold": [],
                "snippet": [],
                "warning": [],
                "buggy": [],
                "predicted_buggy": [],
                "predicted_targets": [],
                "predicted_explanation": [],
            }
            model2_output_data = {
                "fold": [],
                "snippet": [],
                "warning": [],
                "buggy": [],
                "predicted_buggy": [],
                "predicted_targets": [],
                "predicted_explanation": [],
            }

            list_results["list_model1_output_data"][fold] = model1_output_data
            list_results["list_model2_output_data"][fold] = model2_output_data


            buggy_folds = folds[f"{bug_type}_buggy"]
            clean_folds = folds[f"{bug_type}_clean"]

            #print(f"\t\tFOLD: {fold}")
            validation_data = validation_data_list[fold]
            
            index = 0
            #for index, validation_example in enumerate(validation_data):
            while index < len(validation_data):
                if (need_continue_index):
                    index = experiment_progress["index"]
                    need_continue_index = False
                #end continuation check
                experiment_progress["index"] = index
                # example_pairs = self.get_pair_examples(
                #     fold, buggy_folds, clean_folds, 3
                # )
                validation_example = validation_data[index]

                #reminder: pair_examples_list = {"one-shot": [0]*number_of_folds, "few-shot":[0]*number_of_folds}
                example_pairs = pair_examples_list[strategy][fold]
                

                examples = list_examples[fold][index] 



                example_prompt = ChatPromptTemplate.from_messages(
                    [
                        ("human", "{input}"),
                        ("ai", "{output}"),
                    ]
                )

                few_shot_prompt = FewShotChatMessagePromptTemplate(
                    example_prompt=example_prompt,
                    examples=examples,
                )

                prompt_template = ChatPromptTemplate.from_messages(
                    [
                        ("system", "{input_system_prompt}"),
                        few_shot_prompt,
                        ("human", "####{input_code}####"),
                    ]
                )
                #print(f"\t\t\tValidation Example Index: {index}")
                validation_example_snippet = validation_example["snippet"]
                validation_example_warning = validation_example["warning"]
                validation_example_target = validation_example["target"]

                #llm_model1 = self.chatgpt4
                #llm_model2 = self.chatgpt3
                llm_model1 = deepseek1b
                llm_model2 = deepseek1b

                #chain_model1 = LLMChain(llm=llm_model1, prompt=prompt_template) changed this cuz this is old
                chain_model1 = prompt_template | llm_model1 | StrOutputParser()
                response_model1 = chain_model1.invoke(
                    {
                        "input_system_prompt": system_prompt,
                        "input_code": validation_example_snippet,
                    }
                )
                #response_model1 = 'dkalwjdnasdnwakljdnawl json{"result":"dummy1", "targets":[], "explanation":"dummy1explanation"}'

                try:
                    # output_dict_model1 = output_parser.parse(
                    #     response_model1["text"]
                    # )
                    text = response_model1
                    text = custom_clean_json(text) 
                    output_dict_model1 = json.loads(text)
                    #this replaces original parser
                    predicted_result1 = output_dict_model1["result"]
                    predicted_targets1 = output_dict_model1["targets"]
                    predicted_explanation1 = output_dict_model1["explanation"]
                except Exception as e:
                    #print(e)
                    #print(f"\t\t\tJSON Parse Error")
                    predicted_result1 = "fail"
                    predicted_targets1 = "fail"
                    predicted_explanation1 = response_model1

                # adding the results to the model output dictionary for producing the results CSV
                #print(f"\t\t\t{llm_model1.model} has processed the example {index}")

                list_results["list_model1_output_data"][fold]["fold"].append(fold)
                list_results["list_model1_output_data"][fold]["snippet"].append(validation_example_snippet)
                list_results["list_model1_output_data"][fold]["warning"].append(validation_example_warning)
                list_results["list_model1_output_data"][fold]["buggy"].append(validation_example_target)
                list_results["list_model1_output_data"][fold]["predicted_buggy"].append(predicted_result1)
                list_results["list_model1_output_data"][fold]["predicted_targets"].append(predicted_targets1)
                list_results["list_model1_output_data"][fold]["predicted_explanation"].append(
                    predicted_explanation1
                )

                #changed this line also cuz previously was old
                chain_model2 = prompt_template | llm_model2 | StrOutputParser()
                response_model2 = chain_model2.invoke(
                    {
                        "input_system_prompt": system_prompt,
                        "input_code": validation_example_snippet,
                    }
                )

                #response_model2 = 'dkalwjdnasdnwakljdnawl json{"result":"dummy2", "targets":[], "explanation":"dummy2explanation"}'

                try:
                    # output_dict_model2 = output_parser.parse(
                    #     response_model2["text"]
                    # )
                    text = response_model2
                    text = custom_clean_json(text) 
                    output_dict_model2 = json.loads(text)
                    #this replaces original parser
                    predicted_result2 = output_dict_model2["result"]
                    predicted_targets2 = output_dict_model2["targets"]
                    predicted_explanation2 = output_dict_model2["explanation"]
                except Exception as e:
                    #print(e)
                    #print(text)
                    #print(f"\t\t\t>> JSON Parse Error")
                    predicted_result2 = "fail"
                    predicted_targets2 = "fail"
                    predicted_explanation2 = response_model2

                # adding the results to the model output dictionary for producing the results CSV
                #print(f"\t\t\t{llm_model2.model} has processed the example {index}")

                list_results["list_model2_output_data"][fold]["fold"].append(fold)
                list_results["list_model2_output_data"][fold]["snippet"].append(validation_example_snippet)
                list_results["list_model2_output_data"][fold]["warning"].append(validation_example_warning)
                list_results["list_model2_output_data"][fold]["buggy"].append(validation_example_target)
                list_results["list_model2_output_data"][fold]["predicted_buggy"].append(predicted_result2)
                list_results["list_model2_output_data"][fold]["predicted_targets"].append(predicted_targets2)
                list_results["list_model2_output_data"][fold]["predicted_explanation"].append(
                    predicted_explanation2
                )

                #print(f"\t\t\t----------------------------------------------")
                
                print(f"completed experiment {fold} - {index}") 

                # after each completed example, save progress
                # note that this only runs once both models completes. avoiding issue where one is complete and the other get stuck 
                with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl", 'wb') as f:
                    pickle.dump(list_results, f)
                    #print(f"cached /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_part.pkl")
                # save progress json  
                with open(f"/home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_progress.json", 'w') as f:
                    #print(experiment_progress)
                    json.dump(experiment_progress, f)
                    #print(f"progress logged /home/hhjww/Downloads/effectivenessPackage/src/caching/{bug_type}_{strategy}_experiment_progress.json")
                # end of caching and logging  

                #time.sleep(1)
                index += 1
            #end while index
            
            save_results(
                pd.DataFrame(list_results["list_model1_output_data"][fold]),
                experiment,
                bug_type,
                strategy,
                llm_model1.model,
                fold,
            )

            save_results(
                pd.DataFrame(list_results["list_model2_output_data"][fold]),
                experiment,
                bug_type,
                strategy,
                llm_model2.model,
                fold,
            )
            fold += 1
        #end while fold 
        # if fold finishes, means the whole list of examples are completed. write the examples_complete file. 
        with open(f"/home/hhjww/Downloads/effectivenessPackage/src/results/{bug_type}_{strategy}_experiment_complete.pkl", 'wb') as f:
            pickle.dump(list_results, f)
            print(f"cached /home/hhjww/Downloads/effectivenessPackage/src/results/{bug_type}_{strategy}_experiment_complete.pkl")
            print("all result cache completed")
            complete_flag = True
    except OllamaTimeout as e:
        print("ollama timed out restarting")
